Use of Bidirectional_LSTM 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding , LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.utils as k
import numpy as np

In [ ]:
token = Tokenizer()
data = open('i.txt').read()
corpus = data.lower().split('\n')
token.fit_on_texts(corpus)
total_words = len(token.word_index) + 1

In [ ]:
input_seq =[]
for line in corpus:
    token_list = token.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequences = token_list[:i+1]
        input_seq.append(n_gram_sequences)

In [ ]:
max_sequence_len = max([len(x) for x in input_seq])
input_sequences = np.array(pad_sequences(input_seq, maxlen=max_sequence_len , padding='pre'))

In [ ]:
X_train , Y_train = input_sequences[: , :-1] , input_sequences[: , -1]
Y_train = k.to_categorical(Y_train, num_classes = total_words)

In [ ]:
model = Sequential()
model.add(Embedding(total_words ,1000 , input_length = max_sequence_len-1))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dense(total_words / 2 , activation='relu'))
model.add(Dense(total_words , activation = 'softmax'))
model.compile(loss='categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
print(model.summary())

In [ ]:
history = model.fit(X_train , Y_train , epochs  = 15  , verbose = 1)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(len(acc))
plt.plot(epochs , acc , 'b' , label = 'Traning_accuracy')
plt.title('Tranning_accuracy')
plt.figure()
plt.plot(epochs , loss , 'b' , label = 'Traning_accuracy')
plt.title('Traning_loss')
plt.figure()

In [ ]:
text = 'you know me well'
next_word = 200

In [ ]:
for _ in range(next_word):
    token_list = token.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen = max_sequence_len , padding = 'pre')
    predicted = model.predict_classes(token_list , verbose = 0)
    for word , index in token.word_index.items():
        if index == predicted:
            output = word
            break

    text += ' '+output

print(text)